<a href="https://colab.research.google.com/github/maya609/DS2002/blob/main/Finance_API_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd

pd.set_option('display.max_columns', None)

stocks = input("Input two or more stock symbols (separate each symbol with a comma. Ex. AAPL,MSFT): ").strip().replace(" ","").upper()

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stocks}
apikey="1Q9VWASzM07N6ZTmx7jnm8WD4rW9NjEw2SLM0inN"

headers = {
    'x-api-key': apikey
    }
response = requests.request("GET", url, headers=headers, params=querystring)

df = pd.DataFrame(response.json()['quoteResponse']['result'])
df = df[['symbol', 'shortName', 'regularMarketPrice']].rename(
    columns={'symbol': 'Stock Symbol', 'shortName': 'Company Name', 'regularMarketPrice': 'Market Price'}
)
df['Market Price'] = df['Market Price'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
print(df[['Stock Symbol','Company Name','Market Price']])



Input two or more stock symbols (separate each symbol with a comma. Ex. AAPL,MSFT): AAPL,MSFT
  Stock Symbol           Company Name Market Price
0         AAPL             Apple Inc.      $240.36
1         MSFT  Microsoft Corporation      $399.73


In [ ]:
import requests
import json
import pandas as pd


stocks = input("Enter stock symbols separated by commas (Ex. AAPL, MSFT, TSLA): ").strip().replace(" ", "").upper()

modules = {
    "1": "summaryDetail",
    "2": "financialData"
}

print("\nChoose a module:")
print("1 - Summary Detail (52 Week High, 52 Week Low)")
print("2 - Financial Data (Return on Assets)")
choice = input("Enter the number corresponding to your choice: ").strip()

if choice not in modules:
    print("Invalid choice. Exiting.")
    exit()

selected_module = modules[choice]

url = f"https://yfapi.net/v11/finance/quoteSummary/{stocks}"
apikey = "1Q9VWASzM07N6ZTmx7jnm8WD4rW9NjEw2SLM0inN"

params = {"modules": selected_module}
headers = {"x-api-key": apikey}

response = requests.get(url, headers=headers, params=params)
data = response.json()

if not data.get('quoteSummary') or not data['quoteSummary'].get('result'):
    print("No data found for the given symbols.")
    exit()

results = []
for stock_data in data['quoteSummary']['result']:
    stock = stocks
    if selected_module == "summaryDetail":
        details = stock_data.get('summaryDetail', {})
        extracted_data = {
            "Stock Symbol": stock,
            "52 Week High": details.get('fiftyTwoWeekHigh', {}).get('raw', "N/A"),
            "52 Week Low": details.get('fiftyTwoWeekLow', {}).get('raw', "N/A")
        }
    elif selected_module == "financialData":
        financials = stock_data.get('financialData', {})
        extracted_data = {
            "Stock Symbol": stock,
            "Return on Assets (ROA)": financials.get('returnOnAssets', {}).get('raw', "N/A")
        }

    results.append(extracted_data)

df = pd.DataFrame(results)
print("\n", df)

Enter stock symbols separated by commas (Ex. AAPL, MSFT, TSLA): MSFT

Choose a module:
1 - Summary Detail (52 Week High, 52 Week Low)
2 - Financial Data (Return on Assets)
Enter the number corresponding to your choice: 2

   Stock Symbol  Return on Assets (ROA)
0         MSFT                 0.14649


In [11]:
import requests
import pandas as pd

apikey = "1Q9VWASzM07N6ZTmx7jnm8WD4rW9NjEw2SLM0inN"
headers = {"x-api-key": apikey}

trending_url = "https://yfapi.net/v1/finance/trending/US"
response = requests.get(trending_url, headers=headers)

if response.status_code != 200:
    print("Error fetching trending stocks:", response.status_code)
else:
    data = response.json()

    if 'finance' in data and 'result' in data['finance'] and data['finance']['result']:
        trending_tickers = [stock['symbol'] for stock in data['finance']['result'][0]['quotes'][:5]]

        results = []

        for ticker in trending_tickers:
            quote_url = f"https://yfapi.net/v11/finance/quoteSummary/{ticker}"
            params = {"modules": "summaryDetail,price"}

            response = requests.get(quote_url, headers=headers, params=params)

            if response.status_code == 200:
                quote_data = response.json()

                if 'quoteSummary' in quote_data and quote_data['quoteSummary'].get('result'):
                    stock_info = quote_data['quoteSummary']['result'][0]
                    details = stock_info.get('summaryDetail', {})
                    price_info = stock_info.get('price', {})

                    company_name = price_info.get('longName', "N/A")
                    market_price = price_info.get('regularMarketPrice', {}).get('raw', "N/A")

                    extracted_data = {
                        "Stock Symbol": ticker,
                        "Company Name": company_name,
                        "Market Price": market_price,
                        "52 Week High": details.get('fiftyTwoWeekHigh', {}).get('raw', "N/A"),
                        "52 Week Low": details.get('fiftyTwoWeekLow', {}).get('raw', "N/A")
                    }
                    results.append(extracted_data)
                else:
                    print(f"No data found for {ticker}")
            else:
                print(f"Error fetching data for {ticker}: {response.status_code}")

        df = pd.DataFrame(results)
        if df.empty:
            print("No stock data available.")
        else:
            print("\nTrending Stocks Data:\n", df)

    else:
        print("No trending stock data found.")



Trending Stocks Data:
   Stock Symbol        Company Name  Market Price  52 Week High  52 Week Low
0         NVDA  NVIDIA Corporation        131.28        153.13       75.606
1          CRM    Salesforce, Inc.        307.33        369.00      212.000
2         SNOW      Snowflake Inc.        166.19        194.40      107.130
3         IONQ          IonQ, Inc.         29.93         54.74        6.220
4         BYND   Beyond Meat, Inc.          3.56         11.70        3.300
